In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [15]:
get_current_weather("Boston, MA")

'{"location": "Boston, MA", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [6]:
# Full function definition from the above
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]


In [7]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [8]:
import  openai

In [9]:
response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo-0613",
    messages = messages,
    functions = functions
)

print (response)

{
  "id": "chatcmpl-8E0qHsH9qiQX8ZbwTAkVy1PVEgdOE",
  "object": "chat.completion",
  "created": 1698350521,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n\"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 17,
    "total_tokens": 99
  }
}


In [10]:
response_message = response["choices"][0]["message"]
response_message

<OpenAIObject at 0x117dbd2b0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n\"location\": \"Boston, MA\"\n}"
  }
}

In [11]:
response_message["content"]

# there  is no content

In [12]:
response_message["function_call"]

# instead of content we have some 

<OpenAIObject at 0x117dbfa70> JSON: {
  "name": "get_current_weather",
  "arguments": "{\n\"location\": \"Boston, MA\"\n}"
}

In [14]:
json.loads(response_message["function_call"]["arguments"])

{'location': 'Boston, MA'}

In [16]:
args = json.loads(response_message["function_call"]["arguments"])

Function, calling with openAI doesn’t directly call the function. We still have to do that ourselves. Rather it just tells us what function to call that’s the name and what the arguments to that function should be. 

In [17]:
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

Although this is trained to return a (json) block, it’s actually not strictly enforced so if you’re decoding it with Jason and you run into some error, that’s actually an issue on the model side and so you might wanna put in some safeguards around that.

### What happens if the message that you pass in isn’t related to the function at all?

When we call it, we can see the message that’s returned has Content as normal, and it doesn’t have a function parameter.
What’s going on under the hood is that the model is determining whether to use a function or not.
There are additional perimeters that we can pass in to force the model to use or not to use a function like <b>function_call</b> parameter. 

In [19]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [22]:
response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo-0613',
    messages = messages,
    functions = functions
)

print(response)

{
  "id": "chatcmpl-8E1xgH3E1amW2yJjO7xniUK77kJb4",
  "object": "chat.completion",
  "created": 1698354824,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  }
}


In [23]:
# use auto function_call mode

messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",   # Let LLM to choose it which is also the default value
)
print(response)

{
  "id": "chatcmpl-8E22pmqRGOdT0Ynxisa03mEHV1FZY",
  "object": "chat.completion",
  "created": 1698355143,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 10,
    "total_tokens": 86
  }
}


In [25]:
# Force NOT to call function

messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

{
  "id": "chatcmpl-8E29xigCJth9jtIwjS2A535NlcW2a",
  "object": "chat.completion",
  "created": 1698355585,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Please give me a moment. I will check the weather in Boston for you."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 16,
    "total_tokens": 98
  }
}


In [26]:
# force to call function

messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8E2GDvBy8KREnyxA1gt3qklLmhlN6",
  "object": "chat.completion",
  "created": 1698355973,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 89,
    "completion_tokens": 11,
    "total_tokens": 100
  }
}


In [27]:
# force to call function for a message that does NOT require function_call

messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

{
  "id": "chatcmpl-8E2HamDQ6F1TP6wFIeZpseebPAXmJ",
  "object": "chat.completion",
  "created": 1698356058,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}"
        }
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 12,
    "total_tokens": 95
  }
}


We see the language model got confused and made things up.


<br/>

###  How you can pass some of these function calls and the results of actually doing the function calls back into the language model?
This is important because often times you want to use the language model to determine what function to call then run that function, then pass it back into the language model to get a final response.

In [28]:
messages.append(response["choices"][0]["message"])

In [29]:
args = json.loads(response["choices"][0]["message"]["function_call"]["arguments"])
observation = get_current_weather(args)

In [30]:
messages.append(
    {
        "role":"function",
        "name": "get_current_weather",
        "content": observation,
     }
)

In [ ]:
response